In [5]:
import os

In [3]:
prev_dir=os.getcwd()

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\Muhammad Hasnain\\Desktop\\KidneyDiseaseClassification'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class ModelTrainingCofigEntity:
    base_model_path:Path
    trained_model_path:Path
    dataset_path:Path
    params_is_augmented:bool
    params_epochs: int
    params_batch_size: int
    params_img_shape: list 
    params_learning_rate: float 
    params_weights: list




In [10]:
from cnn_classifier.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnn_classifier.utils.common import  read_yaml

In [24]:
class ModelTrainingConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        os.makedirs(self.config.model_training.root_dir,exist_ok=True)

    
    def get_model_config(self):
       return ModelTrainingCofigEntity(base_model_path=Path(self.config.prepare_base_model.updated_base_model_path),trained_model_path=Path(self.config.model_training.trained_model_path),dataset_path=Path(self.config.data_ingestion.unzip_dir+"/kidney_images"),params_epochs=self.params.EPOCHS,params_batch_size=self.params.BATCH_SIZE,params_learning_rate=self.params.LEARNING_RATE,params_img_shape=self.params.IMAGE_SHAPE,params_weights=self.params.WEIGHTS,params_is_augmented=self.params.AUGMENTATION)
        

In [25]:
OBJ= ModelTrainingConfigurationManager()
OBJ.get_model_config()

[2025-09-29 15:57:49,702] : common: INFO: yaml file 'config\config.yaml' loaded 
[2025-09-29 15:57:49,728] : common: INFO: yaml file 'params.yaml' loaded 


ModelTrainingCofigEntity(base_model_path=WindowsPath('artifacts/prepare_base_model/updated_base_model.h5'), trained_model_path=WindowsPath('artifacts/trained_model/model.h5'), dataset_path=WindowsPath('artifacts/data_ingestion/kidney_images'), params_is_augmented=True, params_epochs=1, params_batch_size=16, params_img_shape=BoxList([224, 224, 3]), params_learning_rate=0.01, params_weights='imagenet')

In [34]:
import tensorflow as tf
import numpy as np

In [37]:
class ModelTraining:
    def __init__(self,config:ModelTrainingConfigurationManager):
        self.config=config.get_model_config()
        

    
    def get_base_model(self):
        model=tf.keras.models.load_model(self.config.base_model_path)
        return model


    def train_valid_generator(self):
         self.model=self.get_base_model()
         datagenerator_kwargs = dict( rescale = 1./255, validation_split=0.20 ) 
         dataflow_kwargs = dict( target_size=self.config.params_img_shape[:-1], batch_size=self.config.params_batch_size, interpolation="bilinear" ) 
         valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
              **datagenerator_kwargs )
         
         self.valid_generator = valid_datagenerator.flow_from_directory( 
             directory=self.config.dataset_path, 
             subset="validation", 
             shuffle=False, **dataflow_kwargs ) 
         
         if self.config.params_is_augmented: 
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator( 
                rotation_range=40,
                  horizontal_flip=True,
                    width_shift_range=0.2, 
                    height_shift_range=0.2, 
                    shear_range=0.2, 
                    zoom_range=0.2, 
                    **datagenerator_kwargs ) 
         else: 
            train_datagenerator = valid_datagenerator 
         self.train_generator = train_datagenerator.flow_from_directory( 
                directory=self.config.dataset_path,shuffle=True,
                    **dataflow_kwargs )
             
    @staticmethod 
    def save_model(path: Path, model: tf.keras.Model): 
        model.save(path)
        
    def train(self):
            
            self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size 
            self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size      
            self.model.fit( self.train_generator, epochs=self.config.params_epochs, validation_data=self.valid_generator ) 
            self.save_model( path=self.config.trained_model_path, model=self.model )



        

In [38]:
conf_obj=ModelTrainingConfigurationManager()
obj2=ModelTraining(conf_obj)
obj2.train_valid_generator()
obj2.train()

[2025-09-29 16:01:30,113] : common: INFO: yaml file 'config\config.yaml' loaded 
[2025-09-29 16:01:30,133] : common: INFO: yaml file 'params.yaml' loaded 
[2025-09-29 16:01:30,634] : saving_utils: WARNING: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Found 1471 images belonging to 2 classes.
Found 7360 images belonging to 2 classes.


NotImplementedError: numpy() is only available when eager execution is enabled.

In [23]:
model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 1000)           │     4,097,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)